In [1]:
import numpy as np
import os

In [2]:
folders = ('900023201__20200710_080000', '900100003__20200710_080000')
res_folder = '900100003_900023201__20200710_080000'
grid_x = np.load('{}/grid_x.npy'.format(folders[0]))
grid_y = np.load('{}/grid_y.npy'.format(folders[0]))
points = np.load('{}/points.npy'.format(folders[0]))
if not os.path.exists(res_folder):
    os.makedirs(res_folder)
distance_map = np.load('{}/distance_map.npy'.format(folders[0]))
for folder in folders[1:]:
    distance_map += np.load('{}/distance_map.npy'.format(folder))
distance_map /= len(folders)

In [3]:
bottomleft = points.min(axis=0)
topright = points.max(axis=0)
topleft = topright[0], bottomleft[1]
bottomright = bottomleft[0], topright[1]

Render aggregated map

In [4]:
import folium
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

%matplotlib inline

In [5]:
top = cm.get_cmap('Oranges_r', 128)
bottom = cm.get_cmap('Greens', 128)
newcolors = np.vstack((top(np.linspace(0.2, 0.8, 128)),
                       bottom(np.linspace(0.2, 0.8, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

In [6]:
map_center = (topleft[0] + bottomright[0])/2.0, (topleft[1] + bottomright[1])/2.0

In [7]:
m = folium.Map(map_center, zoom_start=14)
colors = ['green', 'orange', 'red']

overlay = distance_map
norm_factor = np.percentile(overlay,95)
norm_fun = np.vectorize(lambda x: 1 - min(x/norm_factor, 1))
overlay = norm_fun(overlay)

[min_lat, min_lon] = bottomright[0], topleft[1]
[max_lat, max_lon] = topleft[0], bottomright[1]

folium.raster_layers.ImageOverlay(
    image=overlay,
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    colormap=newcmp,
    vmin=0,
    vmax=3600,
    opacity=0.7,
    pixelated=False,
    origin='lower'
).add_to(m)

# enable for debugging
# shape = grid_x.shape
# for col_index in range(0,shape[1],5):
#     for row_index in range(0,shape[0],5):
#         point = GPSPoint(grid_x[col_index][row_index], grid_y[col_index][row_index])
#         distance_map_value = distance_map[col_index][row_index]
#         folium.CircleMarker((point.lat, point.lon), fill_color='black', opacity=0.8, radius=3, color='black', stroke=True, tooltip=distance_map_value).add_to(m)

m.save(os.path.join(res_folder, 'distance_map.html'))

Geocoding example

In [8]:
import urllib.parse
import requests

In [17]:
address = "Frankfurter Allee 15, 10247"
# address = "Pfuelstraße 1"
query = urllib.parse.quote_plus(address)
url = "https://nominatim.openstreetmap.org/?addressdetails=1&q={}&format=json&limit=1".format(query)
response = requests.get(url)

In [18]:
js = response.json()
geo_point = (float(js[0]['lat']), float(js[0]['lon']))

In [19]:
def get_closest_grid_point(grid, point):
    grid_x, grid_y = grid
    search_arr = (grid_x - point[0])**2 + (grid_y - point[1])**2
    min_element = np.argmin(np.abs(search_arr))
    return min_element

def get_distance_for_geopoint(grid, point, distance_map):
    index = get_closest_grid_point((grid_x, grid_y), point)
    return distance_map.ravel()[index]

In [20]:
get_distance_for_geopoint((grid_x,grid_y), geo_point, distance_map)

1078.3054494935707